In [1]:
from turtle import width
import torch
from PIL import Image
from torch.utils.data import DataLoader
import numpy as np

In [2]:
class ImagePixelDatasetHigherDimention(torch.utils.data.Dataset):

  def __init__(self,image_path,L):
    self.image = np.array(Image.open(image_path))
    self.height,self.width,_ = self.image.shape
    self.L = L


  def __len__(self):
    return self.height * self.width

  def __getitem__(self, idx):

    y = idx // self.width
    x = idx % self.width
    color = self.image[y,x]/255

    y = np.interp(y,[0,self.height],[-1,1])
    x = np.interp(x,[0,self.width],[-1,1])

    input = []

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * x))
        input.append(np.cos(2**l * np.pi * x))

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * y))
        input.append(np.cos(2**l * np.pi * y))


    return np.array(input), color

In [3]:
# Example usage
input_image_path = "/content/drive/My Drive/sifnos-greece-3840x2160-12799.jpg"
output_double_resolution = '/content/drive/My Drive/double_resolution.jpg'
output_add_border = '/content/drive/My Drive/add_border.jpg'

border_size = 200

In [4]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
from PIL import Image

def double_resolution(input_path, output_path):
    # Open the image
    original_image = Image.open(input_path)

    # Get the original width and height
    original_width, original_height = original_image.size

    # Create a new image with double the resolution
    new_width = original_width * 2
    new_height = original_height * 2
    new_image = original_image.resize((new_width, new_height), Image.BICUBIC)

    # Save the new image
    new_image.save(output_path)

In [6]:
from PIL import Image, ImageOps

def add_border(input_path, output_path, border_size):
    # Open the image
    original_image = Image.open(input_path)

    # Add a border to the image
    bordered_image = ImageOps.expand(original_image, border=border_size, fill='black')

    # Save the result
    bordered_image.save(output_path)

In [7]:

import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return x

class MLP3(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, output_size):
        super(MLP3, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        return x

class MLP4(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3, output_size):
        super(MLP4, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        return x

In [8]:
model_hd_l2 = torch.load('/content/drive/My Drive/model_mlp_hd_l2.pt')
model_hd_l2.eval()

MLP4(
  (fc1): Linear(in_features=8, out_features=256, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=64, out_features=3, bias=True)
  (relu4): ReLU()
)

In [9]:
double_resolution(input_image_path,output_double_resolution)

In [ ]:
from tqdm import tqdm

L=2

dataset = ImagePixelDatasetHigherDimention(output_double_resolution,L)
coor = [dataset[i][0] for i in tqdm(range(len(dataset)))]
coor = torch.tensor(coor)

model_hd_l2.eval()

batch_size = 131072
with torch.no_grad():
  pre = []
  for i in tqdm(range(0, coor.size(0), batch_size)):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_hd_l2(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_model_hd_double_resolution.png')

100%|██████████| 33177600/33177600 [12:09<00:00, 45478.59it/s]
<ipython-input-10-1fb8b26fa0b8>:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coor = torch.tensor(coor)
  0%|          | 0/254 [00:00<?, ?it/s]<ipython-input-10-1fb8b26fa0b8>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
100%|██████████| 254/254 [00:58<00:00,  4.37it/s]


In [ ]:
add_border(input_image_path,output_add_border,200)

In [10]:
from tqdm import tqdm

L=2

model_hd_l2.eval()

dataset = ImagePixelDatasetHigherDimention(output_add_border,L)
coor = [dataset[i][0] for i in tqdm(range(len(dataset)))]
coor = torch.tensor(coor)

model_hd_l2.eval()

batch_size = 4096
with torch.no_grad():
  pre = []
  for i in tqdm(range(0, coor.size(0), batch_size)):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_hd_l2(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_model_hd_l2_border.png')

100%|██████████| 10854400/10854400 [06:13<00:00, 29074.80it/s]
<ipython-input-10-ff19d60a907f>:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coor = torch.tensor(coor)
  0%|          | 0/2650 [00:00<?, ?it/s]<ipython-input-10-ff19d60a907f>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
100%|██████████| 2650/2650 [00:33<00:00, 78.61it/s]
